# Google Places API Data Collection
This notebook collects data about food establishments in London using the Google Places API.

## Setup Requirements
- Google Maps API key in `.env` file
- Required libraries: python-dotenv, googlemaps, pandas

In [2]:
# Cell 1: Import Libraries
import os
import time
from datetime import datetime
import logging
from dotenv import load_dotenv
import googlemaps
import pandas as pd
import argparse
from typing import Optional, Dict, List
import sys

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='places_api.log'
)

# Load environment variables
load_dotenv()

True

In [3]:
# Cell 2: Define PlacesDataCollector Class for Deliveroo Driver Focus
class PlacesDataCollector:
    def __init__(self):
        """Initialize the PlacesDataCollector with Google Maps client."""
        self.api_key = os.getenv('GOOGLE_MAPS_API_KEY')
        if not self.api_key:
            raise ValueError("Google Maps API key not found in environment variables")
        
        self.gmaps = googlemaps.Client(key=self.api_key)
        self.place_types = [
            'restaurant',           
            'meal_takeaway',        
            'fast_food_restaurant', 
            'food',                
            'meal_delivery',        
            'cafe',
            'grocery_or_supermarket',
            'supermarket',
            'convenience_store'                 
        ]
        
        # Add borough mappings to the PlacesDataCollector
        self.borough_mappings = {
            # Westminster postcodes
            'W1': 'City of Westminster',
            'W2': 'City of Westminster',
            'W9': 'City of Westminster',
            'SW1': 'City of Westminster',
            'NW1': 'City of Westminster',
            'NW8': 'City of Westminster',
            # Specific Westminster postcodes
            'W1A': 'City of Westminster',
            'W1B': 'City of Westminster',
            'W1C': 'City of Westminster',
            'W1D': 'City of Westminster',
            'W1F': 'City of Westminster',
            'W1G': 'City of Westminster',
            'W1H': 'City of Westminster',
            'W1J': 'City of Westminster',
            'W1K': 'City of Westminster',
            'W1S': 'City of Westminster',
            'W1T': 'City of Westminster',
            'W1U': 'City of Westminster',
            'W1W': 'City of Westminster',
            # Kensington and Chelsea postcodes
            'SW3': 'Kensington and Chelsea',
            'SW5': 'Kensington and Chelsea',
            'SW7': 'Kensington and Chelsea',
            'SW10': 'Kensington and Chelsea',
            'W8': 'Kensington and Chelsea',
            'W10': 'Kensington and Chelsea',
            'W11': 'Kensington and Chelsea',
            'W14': 'Kensington and Chelsea'
        }

    def get_borough_from_postcode(self, postcode: str) -> str:
        """Determine borough from postcode using mapping."""
        if not postcode:
            return 'Borough Unknown'
            
        # First try the full postcode prefix (before the space)
        prefix = postcode.split(' ')[0] if ' ' in postcode else postcode
        if prefix in self.borough_mappings:
            return self.borough_mappings[prefix]
            
        # Then try the first part of the postcode
        short_prefix = ''.join(filter(str.isalpha, prefix))
        if short_prefix in self.borough_mappings:
            return self.borough_mappings[short_prefix]
            
        return 'Borough Unknown'
        
    def get_location_details(self, postcode: str) -> tuple:
        """Get location details including coordinates and specific borough."""
        try:
            geocode_result = self.gmaps.geocode(postcode)
            if geocode_result:
                location = geocode_result[0]['geometry']['location']
                
                # Use the new borough mapping function
                borough = self.get_borough_from_postcode(postcode)
                
                return location['lat'], location['lng'], borough
            return None, None, None
        except Exception as e:
            logging.error(f"Error geocoding postcode {postcode}: {str(e)}")
            raise

    def get_place_details(self, place_id: str) -> dict:
        """Get detailed information about a specific place."""
        try:
            time.sleep(2)  # Basic rate limiting
            place_details = self.gmaps.place(place_id, fields=['name', 'geometry', 'type', 'opening_hours'])
            return place_details.get('result', {})
        except Exception as e:
            logging.error(f"Error fetching place details for {place_id}: {str(e)}")
            return {}

    def simplify_store_type(self, store_type):
        """
        Simplify store types into main categories:
        - Supermarket for grocery/supermarket/convenience types
        - Takeaway for everything else
        """
        supermarket_types = ['grocery_or_supermarket', 'supermarket', 'convenience_store']
        
        if store_type in supermarket_types:
            return 'Supermarket'
        return 'Takeaway'

    def format_time_HHMM(self, time_str):
        """Convert time from HHMM to HH:MM format."""
        if pd.isna(time_str) or time_str == '':
            return None
        return f"{time_str[:2]}:{time_str[2:]}"

    def format_opening_hours(self, opening_hours: dict) -> list:
        """Format opening hours into required structure."""
        if not opening_hours or 'periods' not in opening_hours:
            return []

        days_data = []
        for period in opening_hours['periods']:
            if 'open' in period and 'close' in period:
                days_data.append({
                    'day': period['open']['day'],
                    'opening_time': period['open']['time'],
                    'closing_time': period['close']['time']
                })
        return days_data

    def collect_places_data(self, postcode: str) -> pd.DataFrame:
        """Collect data for all food establishments in the given postcode."""
        print(f"Starting data collection for {postcode}")
        
        try:
            lat, lng, borough = self.get_location_details(postcode)
            if not lat or not lng:
                raise ValueError(f"Could not get coordinates for postcode {postcode}")

            all_places_data = []
            # Track unique establishments by name AND location
            collected_locations = set()  # Will store tuples of (name, lat, lng)
            
            for place_type in self.place_types:
                print(f"Searching for {place_type}...")
                
                try:
                    places_result = self.gmaps.places_nearby(
                        location=(lat, lng),
                        radius=1000,
                        type=place_type
                    )

                    for place in places_result.get('results', []):
                        place_name = place.get('name', '')
                        place_lat = place.get('geometry', {}).get('location', {}).get('lat')
                        place_lng = place.get('geometry', {}).get('location', {}).get('lng')
                        
                        # Create unique identifier using name and location
                        location_identifier = (place_name, place_lat, place_lng)
                        
                        # Skip if we've already collected this exact establishment
                        if location_identifier in collected_locations:
                            continue
                            
                        collected_locations.add(location_identifier)
                        print(f"Found: {place_name} at ({place_lat}, {place_lng})")
                        place_details = self.get_place_details(place['place_id'])
                        
                        if not place_details:
                            continue
                            
                        opening_hours = place_details.get('opening_hours', {})
                        hours_data = self.format_opening_hours(opening_hours)
                        
                        for hours in hours_data:
                            all_places_data.append({
                                'Name': place_name,
                                'Postcode': postcode,
                                'Borough': borough,
                                'Latitude': place_lat,
                                'Longitude': place_lng,
                                'Store_Type': self.simplify_store_type(place_type),
                                'Day': hours.get('day', ''),
                                'Opening_Hours': self.format_time_HHMM(hours.get('opening_time', '')),
                                'Closing_Hours': self.format_time_HHMM(hours.get('closing_time', ''))
                            })

                    # Handle pagination if necessary
                    while 'next_page_token' in places_result:
                        time.sleep(2)
                        places_result = self.gmaps.places_nearby(
                            location=(lat, lng),
                            radius=1000,
                            type=place_type,
                            page_token=places_result['next_page_token']
                        )
                        
                except Exception as e:
                    logging.error(f"Error processing {place_type}: {str(e)}")
                    continue

            # Convert to DataFrame
            df = pd.DataFrame(all_places_data)
            
            return df

        except Exception as e:
            logging.error(f"Error collecting data for postcode {postcode}: {str(e)}")
            raise

In [4]:
# Cell 3: Define Tableau Animation Function
def prepare_data_for_tableau_animation(df):
    """Prepare the DataFrame for Tableau time-based animation, showing all hours."""
    
    # Create a list to store expanded records
    expanded_records = []
    
    # Base date
    base_date = pd.Timestamp('2025-01-06')  # This is a Monday
    
    # Generate hourly intervals
    time_intervals = pd.date_range("00:00:00", "23:00:00", freq="1H").time
    
    for _, row in df.iterrows():
        # Convert opening and closing times to datetime.time objects
        opening_time = pd.to_datetime(row['Opening_Hours'], format='%H:%M').time()
        closing_time = pd.to_datetime(row['Closing_Hours'], format='%H:%M').time()
        
        # Get day number directly (it's already 0-6)
        day_num = int(row['Day'])
        
        for interval in time_intervals:
            # Handle overnight hours
            if closing_time > opening_time:
                # Normal hours (same day)
                is_open = opening_time <= interval <= closing_time
                time_date = pd.Timestamp.combine(base_date + pd.Timedelta(days=day_num), interval)
            else:
                # Overnight hours
                if opening_time <= interval:
                    # Current day after opening
                    is_open = True
                    time_date = pd.Timestamp.combine(base_date + pd.Timedelta(days=day_num), interval)
                elif interval <= closing_time:
                    # Next day until closing
                    is_open = True
                    time_date = pd.Timestamp.combine(base_date + pd.Timedelta(days=day_num + 1), interval)
                else:
                    # Outside of opening hours
                    is_open = False
                    time_date = pd.Timestamp.combine(base_date + pd.Timedelta(days=day_num), interval)
                    # Also create next day early morning records
                    if interval <= closing_time:
                        expanded_records.append({
                            'Name': row['Name'],
                            'Postcode': row['Postcode'],
                            'Borough': row['Borough'],
                            'Latitude': row['Latitude'],
                            'Longitude': row['Longitude'],
                            'Store_Type': row['Store_Type'],
                            'Day': row['Day'],
                            'Opening_Hours': row['Opening_Hours'],
                            'Closing_Hours': row['Closing_Hours'],
                            'Time_Date': pd.Timestamp.combine(base_date + pd.Timedelta(days=day_num + 1), interval),
                            'Is_Open': interval <= closing_time
                        })
            
            expanded_records.append({
                'Name': row['Name'],
                'Postcode': row['Postcode'],
                'Borough': row['Borough'],
                'Latitude': row['Latitude'],
                'Longitude': row['Longitude'],
                'Store_Type': row['Store_Type'],
                'Day': row['Day'],
                'Opening_Hours': row['Opening_Hours'],
                'Closing_Hours': row['Closing_Hours'],
                'Time_Date': time_date,
                'Is_Open': is_open
            })
    
    # Create new DataFrame with expanded time intervals
    df_expanded = pd.DataFrame(expanded_records)
    
    # Sort by establishment name and Time_Date
    df_expanded = df_expanded.sort_values(['Name', 'Time_Date'])
    
    return df_expanded

In [5]:
# Cell 4: PostcodeProcessor Class - Core Data Collection
class PostcodeProcessor:
    def __init__(self, timestamp, output_dir):
        """Initialize with complete postcodes for Westminster and Kensington & Chelsea."""
        # Westminster postcodes mapping - key areas with borough assignment
        self.westminster_postcodes_map = {
            'W1K 6TL': 'City of Westminster', 'W1S 1YQ': 'City of Westminster',
            'W1B 5AH': 'City of Westminster', 'W1F 7TR': 'City of Westminster',
            'W1T 7NN': 'City of Westminster', 'W1U 3QA': 'City of Westminster',
            'W1G 8PY': 'City of Westminster', 'W1W 7JE': 'City of Westminster',
            'W1H 7EY': 'City of Westminster', 'W2 1HY': 'City of Westminster',
            'W2 2RL': 'City of Westminster', 'W2 4QJ': 'City of Westminster',
            'W2 6BD': 'City of Westminster', 'NW8 7JN': 'City of Westminster',
            'NW8 9AY': 'City of Westminster', 'NW8 0LH': 'City of Westminster',
            'SW1A 1AA': 'City of Westminster', 'SW1E 5JL': 'City of Westminster',
            'SW1H 9EA': 'City of Westminster', 'SW1P 3EU': 'City of Westminster',
            'SW1V 1RG': 'City of Westminster', 'SW1W 0NY': 'City of Westminster',
            'SW1X 7XL': 'City of Westminster', 'SW1Y 4AB': 'City of Westminster',
            'W9 1AX': 'City of Westminster', 'W9 2AL': 'City of Westminster',
            'W9 3DP': 'City of Westminster'
        }
        
        # Kensington and Chelsea postcodes mapping
        self.chelsea_postcodes_map = {
            'SW3 1ER': 'Kensington and Chelsea', 'SW3 3DW': 'Kensington and Chelsea',
            'SW3 5EL': 'Kensington and Chelsea', 'SW3 6RT': 'Kensington and Chelsea',
            'SW5 9PR': 'Kensington and Chelsea', 'SW5 0SW': 'Kensington and Chelsea',
            'SW5 9QP': 'Kensington and Chelsea', 'SW7 2AZ': 'Kensington and Chelsea',
            'SW7 5BD': 'Kensington and Chelsea', 'SW7 4QL': 'Kensington and Chelsea',
            'SW10 0XE': 'Kensington and Chelsea', 'SW10 9SU': 'Kensington and Chelsea',
            'SW10 0AG': 'Kensington and Chelsea', 'W8 5ED': 'Kensington and Chelsea',
            'W8 7LP': 'Kensington and Chelsea', 'W8 6DN': 'Kensington and Chelsea',
            'W10 6TT': 'Kensington and Chelsea', 'W10 5XL': 'Kensington and Chelsea',
            'W10 4AA': 'Kensington and Chelsea', 'W11 2BQ': 'Kensington and Chelsea',
            'W11 3JQ': 'Kensington and Chelsea', 'W11 1QB': 'Kensington and Chelsea',
            'W14 8TH': 'Kensington and Chelsea', 'W14 9JH': 'Kensington and Chelsea',
            'W14 0RH': 'Kensington and Chelsea'
        }
        
        # Combine for easy lookup
        self.all_postcodes_map = {**self.westminster_postcodes_map, **self.chelsea_postcodes_map}
        
        # Extract lists for processing
        self.westminster_postcodes = list(self.westminster_postcodes_map.keys())
        self.chelsea_postcodes = list(self.chelsea_postcodes_map.keys())
        
        self.collector = PlacesDataCollector()
        self.timestamp = timestamp
        self.output_dir = output_dir
        self.set_filenames()
        
        self.day_map = {
            0: 'Sunday', 1: 'Monday', 2: 'Tuesday', 3: 'Wednesday',
            4: 'Thursday', 5: 'Friday', 6: 'Saturday'
        }
    
    def set_filenames(self, test_mode=False):
        """
        Set filenames based on run mode.
        
        Args:
            test_mode (bool): If True, adds 'test_' prefix to filenames
        """
        mode_prefix = 'test_' if test_mode else ''
        self.raw_filename = os.path.join(
            self.output_dir,
            f"all_establishments_raw_{mode_prefix}{self.timestamp}.csv"
        )

    def deduplicate_establishment_data(self, df):
        """
        Remove duplicate establishments while preserving unique location data.
        
        Args:
            df (pd.DataFrame): DataFrame containing establishment data
            
        Returns:
            pd.DataFrame: Deduplicated data maintaining unique locations
        """
        # Sort by Name, Lat, Long, and Day for consistent selection
        df_sorted = df.sort_values(['Name', 'Latitude', 'Longitude', 'Day'])
        
        # Keep unique establishments based on name and location
        unique_establishments = df_sorted.drop_duplicates(
            subset=['Name', 'Latitude', 'Longitude', 'Day']
        )
        
        return unique_establishments

    def clean_boroughs(self, df: pd.DataFrame) -> pd.DataFrame:
        """Clean borough data using predefined postcode mappings."""
        print(f"Starting borough cleaning on {len(df)} rows")
        print(f"Initial borough distribution:")
        print(df['Borough'].value_counts())
        
        def get_correct_borough(row):
            postcode = row['Postcode']
            current_borough = row['Borough']
            if current_borough == 'Borough Unknown' and postcode in self.all_postcodes_map:
                return self.all_postcodes_map[postcode]
            # If still unknown, try using the collector's mapping
            if current_borough == 'Borough Unknown':
                return self.collector.get_borough_from_postcode(postcode)
            return current_borough
            
        df['Borough'] = df.apply(get_correct_borough, axis=1)
        
        print("\nAfter cleaning:")
        print(df['Borough'].value_counts())
        print(f"Final unique establishments: {len(df['Name'].unique())}")
        return df

    def update_csv_data(self, new_data, filename):
        """
        Update existing CSV with new batch data.
        Handles first write and subsequent updates.
        
        Args:
            new_data (pd.DataFrame): New batch of data to add
            filename (str): Target CSV file
            
        Returns:
            pd.DataFrame: Updated complete dataset
        """
        try:
            if os.path.exists(filename):
                # Read existing data
                existing_data = pd.read_csv(filename)
                # Union with new data
                updated_data = pd.concat([existing_data, new_data], ignore_index=True)
                # Remove any duplicates
                updated_data = self.deduplicate_establishment_data(updated_data)
            else:
                updated_data = new_data
                
            # Save updated data
            updated_data.to_csv(filename, index=False)
            return updated_data
            
        except Exception as e:
            logging.error(f"Error updating CSV {filename}: {str(e)}")
            new_data.to_csv(filename + '.backup', index=False)
            return new_data
        
    def process_existing_data(self, input_filename: str) -> pd.DataFrame:
        """Process existing raw data file instead of collecting new data."""
        try:
            # Read existing data
            raw_df = pd.read_csv(input_filename)
            
            # Clean boroughs
            cleaned_df = self.clean_boroughs(raw_df)
            
            # Save to new file with timestamp
            new_filename = os.path.join(
                self.output_dir,
                f"all_establishments_raw_cleaned_{self.timestamp}.csv"
            )
            cleaned_df.to_csv(new_filename, index=False)
            print(f"Cleaned data saved to: {new_filename}")
            
            return cleaned_df
            
        except Exception as e:
            logging.error(f"Error processing existing file {input_filename}: {str(e)}")
            raise
    
    def process_postcodes(self, batch_size=2, delay_between_batches=600, test_mode=False, 
                        test_postcodes: Optional[List[str]] = None):
        """
        Process postcodes in batches and collect establishment data.
        
        Args:
            batch_size (int): Number of postcodes to process in each batch
            delay_between_batches (int): Delay in seconds between batches
            test_mode (bool): If True, only process test postcodes
            test_postcodes (List[str], optional): Specific postcodes to test
        """
        self.set_filenames(test_mode)
        
        # Determine which postcodes to process
        if test_mode:
            if test_postcodes:
                # Use provided test postcodes
                all_postcodes = test_postcodes
                print("\nRUNNING IN TEST MODE - Processing specified test postcodes:")
                print(f"Test postcodes: {', '.join(all_postcodes)}")
            else:
                # Use default test selection: 3 from each borough
                test_west = self.westminster_postcodes[:3]
                test_chelsea = self.chelsea_postcodes[:3]
                all_postcodes = test_west + test_chelsea
                print("\nRUNNING IN TEST MODE - Processing default test postcodes:")
                print("Westminster postcodes:", ', '.join(test_west))
                print("Chelsea postcodes:", ', '.join(test_chelsea))
        else:
            # Process all postcodes
            all_postcodes = self.westminster_postcodes + self.chelsea_postcodes
        
        batch_number = 1
            
        print(f"Starting processing of {len(all_postcodes)} postcodes")
        print(f"Processing in batches of {batch_size} with {delay_between_batches}s delay between batches")
        print(f"Data will be saved to directory: {self.output_dir}")
        print(f"Raw data file: {os.path.basename(self.raw_filename)}")
        
        for i in range(0, len(all_postcodes), batch_size):
                batch = all_postcodes[i:i + batch_size]
                print(f"\nProcessing batch {batch_number}")
                print(f"Postcodes: {', '.join(batch)}")
                batch_data = []
                
                for postcode in batch:
                    try:
                        print(f"\nProcessing {postcode}")
                        df = self.collector.collect_places_data(postcode)
                        if not df.empty:
                            batch_data.append(df)
                            print(f"Found {len(df)} establishments in {postcode}")
                    except Exception as e:
                        logging.error(f"Error processing postcode {postcode}: {str(e)}")
                        continue
                
                if batch_data:
                    # Combine batch data
                    batch_df = pd.concat(batch_data, ignore_index=True)
                    
                    # Update raw data CSV
                    print(f"\nUpdating raw data CSV...")
                    raw_data = self.update_csv_data(batch_df, self.raw_filename)
                    
                    # Print batch summary
                    print(f"\nBatch {batch_number} Summary:")
                    print(f"Processed postcodes: {', '.join(batch)}")
                    print(f"New establishments: {len(batch_df)}")
                    print(f"Total unique establishments: {len(raw_data['Name'].unique())}")
                    print(f"Remaining postcodes: {len(all_postcodes) - (i + len(batch))}")
                    
                    print("\nCurrent data distribution:")
                    print("\nBy borough:")
                    print(raw_data['Borough'].value_counts())
                    print("\nBy store type:")
                    print(raw_data['Store_Type'].value_counts())
                
                batch_number += 1
                
                if i + batch_size < len(all_postcodes):
                    print(f"\nWaiting {delay_between_batches} seconds before next batch...")
                    time.sleep(delay_between_batches)
            
        print("\nCollection complete!")
        if os.path.exists(self.raw_filename):
            print(f"Raw data available in: {self.raw_filename}")
            return pd.read_csv(self.raw_filename)
        return None

In [6]:
# Cell 5: Tableau Data Preparation Class
class TableauDataPreparator:
    def __init__(self, timestamp, output_dir):
        """
        Initialize Tableau data preparation.
        
        Args:
            timestamp (str): Timestamp for filenames
            output_dir (str): Directory for output files
        """
        self.timestamp = timestamp
        self.output_dir = output_dir
        
        # Set filenames with directory paths
        self.tableau_filename = os.path.join(
            self.output_dir,
            f"all_establishments_tableau_{timestamp}.csv"
        )
        self.validation_filename = os.path.join(
            self.output_dir,
            f"establishments_validation_{timestamp}.csv"
        )
        
        # Base date for time calculations
        self.base_date = pd.Timestamp('2025-01-06')  # Monday
        
        # Day mapping for validation and reporting
        self.day_map = {
            0: 'Sunday',
            1: 'Monday',
            2: 'Tuesday',
            3: 'Wednesday',
            4: 'Thursday',
            5: 'Friday',
            6: 'Saturday'
        }
            
    def prepare_data_for_tableau(self, raw_df):
        """Prepare raw data for Tableau visualization."""
        print(f"Starting Tableau data preparation with {len(raw_df)} rows")
        expanded_records = []
        time_intervals = pd.date_range("00:00:00", "23:00:00", freq="h").time  # Fixed deprecation warning
        # Get unique establishments
        unique_establishments = raw_df.drop_duplicates(subset=['Name', 'Latitude', 'Longitude'])
        print(f"Processing {len(unique_establishments)} unique establishments")
        
        for idx, est in unique_establishments.iterrows():
            if idx % 100 == 0:  # Progress update every 100 establishments
                print(f"Processing establishment {idx} of {len(unique_establishments)}")
                
            # Get all records for this establishment
            est_records = raw_df[
                (raw_df['Name'] == est['Name']) & 
                (raw_df['Latitude'] == est['Latitude']) & 
                (raw_df['Longitude'] == est['Longitude'])
            ]
            
            # Process each day (0-6)
            for day in range(7):
                # Get operating hours for current day and next day
                day_records = est_records[est_records['Day'] == day]
                next_day = (day + 1) % 7
                next_day_records = est_records[est_records['Day'] == next_day]
                
                # Initialize time variables
                opening_time = None
                closing_time = None
                max_closing_hour = None
                is_overnight = False
                
                if not day_records.empty:
                    try:
                        # Get opening and closing times
                        opening_time = pd.to_datetime(day_records.iloc[0]['Opening_Hours'], format='%H:%M').time()
                        closing_time = pd.to_datetime(day_records.iloc[0]['Closing_Hours'], format='%H:%M').time()
                        is_overnight = closing_time < opening_time
                        
                        if is_overnight and not next_day_records.empty:
                            max_closing_hour = pd.to_datetime(next_day_records.iloc[0]['Closing_Hours'], format='%H:%M').time()
                    except (ValueError, TypeError) as e:
                        print(f"Warning: Error processing times for {est['Name']} on day {day}: {str(e)}")
                        continue
                
                # Generate records for each hour
                for interval in time_intervals:
                    is_open = False
                    actual_work_day = day
                    
                    if opening_time and closing_time:
                        if not is_overnight:
                            is_open = opening_time <= interval <= closing_time
                        else:
                            if opening_time <= interval:
                                is_open = True
                            elif max_closing_hour is not None and interval <= max_closing_hour:
                                is_open = True
                                actual_work_day = (day - 1) % 7
                    
                    calendar_datetime = pd.Timestamp.combine(
                        self.base_date + pd.Timedelta(days=day), 
                        interval
                    )
                    
                    actual_work_datetime = pd.Timestamp.combine(
                        self.base_date + pd.Timedelta(days=actual_work_day), 
                        interval
                    )
                    
                    expanded_records.append({
                        'Name': est['Name'],
                        'Postcode': est['Postcode'],
                        'Borough': est['Borough'],
                        'Latitude': est['Latitude'],
                        'Longitude': est['Longitude'],
                        'Store_Type': est['Store_Type'],
                        'Day': day,
                        'Actual_Day_of_Work': actual_work_day,
                        'Opening_Hours': opening_time.strftime('%H:%M') if opening_time else None,
                        'Closing_Hours': closing_time.strftime('%H:%M') if closing_time else None,
                        'Time_Date': calendar_datetime,
                        'Actual_Day_of_Work_DateTime': actual_work_datetime,
                        'Is_Open': is_open
                    })
        
        print("Creating DataFrame from expanded records...")
        df_expanded = pd.DataFrame(expanded_records)
        if len(df_expanded) > 0:
            df_expanded = df_expanded.sort_values(['Name', 'Latitude', 'Longitude', 'Time_Date'])
            print(f"Created DataFrame with {len(df_expanded)} rows for {len(df_expanded['Name'].unique())} establishments")
        else:
            print("Warning: No records were created!")
        
        return df_expanded
    
    
    def update_tableau_data(self, raw_df):
        """
        Update Tableau data file with new processed data.
        """
        try:
        # Process new data
            print("Processing data for Tableau...")
            print(f"Input data contains {len(raw_df['Name'].unique())} unique establishments")
            
            # Before processing
            unique_locations = raw_df.groupby(['Name', 'Latitude', 'Longitude']).size().reset_index()
            print(f"Found {len(unique_locations)} unique location combinations")
            
            new_tableau_data = self.prepare_data_for_tableau(raw_df)
            
            # After processing
            unique_processed = new_tableau_data.groupby(['Name', 'Latitude', 'Longitude']).size().reset_index()
            print(f"Processed {len(unique_processed)} unique locations")
            
            if os.path.exists(self.tableau_filename):
                print("Updating existing Tableau data file...")
                existing_data = pd.read_csv(self.tableau_filename)
                print(f"Existing data has {len(existing_data['Name'].unique())} establishments")
                
                # Union with new data
                updated_data = pd.concat([existing_data, new_tableau_data], ignore_index=True)
                
                # Remove duplicates based on all identifying columns
                updated_data = updated_data.drop_duplicates(
                    subset=['Name', 'Latitude', 'Longitude', 'Day', 'Time_Date']
                )
                print(f"After merging and deduplication: {len(updated_data['Name'].unique())} establishments")
            else:
                print("Creating new Tableau data file...")
                updated_data = new_tableau_data
            
            # Save updated data
            updated_data.to_csv(self.tableau_filename, index=False)
            print(f"Tableau data saved to: {self.tableau_filename}")
            print(f"Final count: {len(updated_data['Name'].unique())} unique establishments")
            
            # Create validation results
            validation_results = self.validate_data(updated_data)
            validation_results.to_csv(self.validation_filename, index=False)
            print(f"Validation results saved to: {self.validation_filename}")
            
            return updated_data
    
        except Exception as e:
            logging.error(f"Error updating Tableau data: {str(e)}")
            print(f"Error during Tableau data update: {str(e)}")
            raise

    

    def validate_data(self, tableau_df):
        """
        Validate the Tableau data for completeness and consistency.
        
        Args:
            tableau_df (pd.DataFrame): Processed Tableau data
            
        Returns:
            pd.DataFrame: Validation results
        """
        validation_results = []
        unique_establishments = tableau_df.drop_duplicates(subset=['Name', 'Latitude', 'Longitude'])
        
        for _, est in unique_establishments.iterrows():
            # Get all records for this establishment
            est_records = tableau_df[
                (tableau_df['Name'] == est['Name']) & 
                (tableau_df['Latitude'] == est['Latitude']) & 
                (tableau_df['Longitude'] == est['Longitude'])
            ]
            
            # Check for complete coverage
            hours_per_day = est_records.groupby('Day').size()
            is_complete = (len(hours_per_day) == 7) and all(count == 24 for count in hours_per_day)
            
            # Check for overnight operations
            has_overnight = any(
                est_records['Day'] != est_records['Actual_Day_of_Work']
            )
            
            validation_results.append({
                'Name': est['Name'],
                'Location': f"({est['Latitude']}, {est['Longitude']})",
                'Store_Type': est['Store_Type'],
                'Borough': est['Borough'],
                'Total_Hours': len(est_records),
                'Complete_Coverage': is_complete,
                'Has_Overnight_Operations': has_overnight,
                'Hours_Per_Day': dict(hours_per_day)
            })
        
        return pd.DataFrame(validation_results)

In [7]:
# Cell 6: Execute Data Collection and Preparation Process
# Initialize timestamp and output directory
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = f"deliveroo_data_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

# Set parameters
test_mode = True  # Set to False for full run
input_file = None  # Set to filename if processing existing data to skip API collection process
test_postcodes = [
    'W1K 6TL', 'SW1A 1AA', 'W1S 1YQ',  # Westminster
    'SW1X 7XL', 'SW1W 0NY', 'SW1P 3EU'     # Chelsea
]

# Initialize processors
processor = PostcodeProcessor(timestamp, output_dir)
tableau_preparator = TableauDataPreparator(timestamp, output_dir)

if input_file:
    print(f"Processing existing file: {input_file}")
    raw_df = processor.process_existing_data(input_file)
else:
    print("Starting collection with continuous CSV updates:")
    print(f"Mode: {'TEST - Selected postcodes only' if test_mode else 'FULL RUN'}")
    if test_mode:
        print("Test postcodes:", test_postcodes)
    print("- Processing 2 postcodes at a time")
    print("- 10 minute break between batches")
    print(f"\nOutput Directory: {output_dir}")
    
    raw_df = processor.process_postcodes(
        batch_size=2, 
        delay_between_batches=600,
        test_mode=test_mode,
        test_postcodes=test_postcodes if test_mode else None
    )

# If data was collected successfully
if raw_df is not None:
    print("\nProcessing final Tableau data...")
    tableau_df = tableau_preparator.update_tableau_data(raw_df)
    
    print("\nFinal Collection Summary:")
    print(f"Total unique establishments: {len(raw_df['Name'].unique())}")
    
    print("\nEstablishments by borough:")
    display(raw_df['Borough'].value_counts())
    
    print("\nEstablishments by type:")
    display(raw_df['Store_Type'].value_counts())
    
    print("\nData time ranges:")
    print("Calendar time range:")
    print(f"Start: {tableau_df['Time_Date'].min()}")
    print(f"End: {tableau_df['Time_Date'].max()}")
    print("\nActual work day time range:")
    print(f"Start: {tableau_df['Actual_Day_of_Work_DateTime'].min()}")
    print(f"End: {tableau_df['Actual_Day_of_Work_DateTime'].max()}")
    
    # Show overnight operations analysis
    overnight_ops = tableau_df[
        tableau_df['Day'] != tableau_df['Actual_Day_of_Work']
    ]['Name'].unique()
    
    if len(overnight_ops) > 0:
        print(f"\nFound {len(overnight_ops)} establishments with overnight operations")
        print("Sample of overnight establishments:")
        sample_df = tableau_df[
            (tableau_df['Name'].isin(overnight_ops[:3])) & 
            (tableau_df['Is_Open'])
        ][['Name', 'Day', 'Actual_Day_of_Work', 'Time_Date', 
            'Actual_Day_of_Work_DateTime', 'Opening_Hours', 'Closing_Hours']]
        display(sample_df.head(10))

print("\nScript execution complete!")
print(f"All output files are in directory: {output_dir}")

Starting collection with continuous CSV updates:
Mode: TEST - Selected postcodes only
Test postcodes: ['W1K 6TL', 'SW1A 1AA', 'W1S 1YQ', 'SW1X 7XL', 'SW1W 0NY', 'SW1P 3EU']
- Processing 2 postcodes at a time
- 10 minute break between batches

Output Directory: deliveroo_data_20250109_010753

RUNNING IN TEST MODE - Processing specified test postcodes:
Test postcodes: W1K 6TL, SW1A 1AA, W1S 1YQ, SW1X 7XL, SW1W 0NY, SW1P 3EU
Starting processing of 6 postcodes
Processing in batches of 2 with 600s delay between batches
Data will be saved to directory: deliveroo_data_20250109_010753
Raw data file: all_establishments_raw_test_20250109_010753.csv

Processing batch 1
Postcodes: W1K 6TL, SW1A 1AA

Processing W1K 6TL
Starting data collection for W1K 6TL
Searching for restaurant...
Found: Claridge's at (51.5125404, -0.1478631)
Found: The Chesterfield Mayfair at (51.5076899, -0.1471358)
Found: The Mandeville Hotel at (51.51647850000001, -0.1509183)
Found: Durrants Hotel at (51.51800189999999, -0.15

KeyboardInterrupt: 